In [8]:
import requests
import re 
from bs4 import BeautifulSoup 
import pandas as pd 
import numpy as np 


data = {} 

listofstates = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

URL = "https://www.nytimes.com/interactive/2020/us/states-reopen-map-coronavirus.html"

page = requests.get(URL) 

soup = BeautifulSoup(page.content, 'html.parser')

mydivs = soup.findAll("div", {"class": "g-inner-box"})


statelist = [] 
reopenedplaceslist = [] 
orderexpirationlist = []
reopenedboollist = [] 

for div in mydivs: 
    state = div.find("div", {"data-state" : re.compile(r".*")})['data-state']
    
    statelist.append(state)
    
    try: 
        reopenedbool = div.find("div", {"class": 'g-details-subhed'}).text
    except: 
        reopenedbool = 'Not Reopened'
    
    
    reopenedboollist.append(reopenedbool)
    
    #grab list of reopened places 
    reopenedplaces = div.find_all("div", {"class": 'g-cat-name'})

    #change list of reopenedplaces to text 
    for i in range(len(reopenedplaces)):
        reopenedplaces[i] = reopenedplaces[i].text
    
    reopenedplaceslist.append(reopenedplaces)
    

    
    orderexpiration = div.find("span", {"class": 'g-date-details-text'}).text
        
    orderexpirationlist.append(orderexpiration)
    

    
df = pd.DataFrame() 
df['State'] = statelist
df['Reopened'] = reopenedbool

df['Reopened Places Overview'] = reopenedplaceslist
df['Order Expiration'] = orderexpirationlist    

df['Outdoor and recreation'] = False 
df['Retail'] = False 
df['Food and drink'] = False 
df['Personal care'] = False 
df['Entertainment'] = False 
df['Industries'] = False 
df['Houses of worship'] = False 



## PLEASE NOTE: MARYLAND IS AN EXCEPTION. do not have as granular as wanted. reopening soon is not its own section

for index, row in df.iterrows():
    templist = row["Reopened Places Overview"]
    if "Outdoor and recreation" in templist: 
        df.iloc[index, df.columns.get_loc("Outdoor and recreation")] = True 

    if "Retail" in templist: 
        df.iloc[index, df.columns.get_loc("Retail")] = True 
    
    if "Food and drink" in templist: 
        df.iloc[index, df.columns.get_loc("Food and drink")] = True 

    if "Personal care" in templist: 
        df.iloc[index, df.columns.get_loc("Personal care")] = True 
    if "Entertainment" in templist: 
        df.iloc[index, df.columns.get_loc("Entertainment")] = True
    if "Industries" in templist: 
        df.iloc[index, df.columns.get_loc("Industries")] = True 
    if "Houses of worship" in templist: 
        df.iloc[index, df.columns.get_loc("Houses of worship")] = True  

    
df["Total Count"] = 0  

df["Total Count"] = 0 
incorrectcount = 0 
for i in range(len(df.index)):
    for j in range(len(df.columns)): 
        if(df.iat[i,j] == True): 
            df.iat[i,11] += 1

            
            
            
import datetime

year = str(datetime.datetime.today().year) 
month = str(datetime.datetime.today().month) 
day = str(datetime.datetime.today().day) 

        
        
df.to_csv('SMB States Reopening NY Times Article ' + year + "." + month + "." + day + '.csv')


# Sandbox

2020
6
18


In [77]:
#reformat with columns for each type of stuff 
reopenedplaceslist

templist = []

for place in reopenedplaceslist: 
    for p in place: 
        if p not in templist: 
            templist.append(p) 
templist 


# ['Outdoor and recreation',
#  'Retail',
#  'Food and drink',
#  'Personal care',
#  'Entertainment',
#  'Industries',
#  'Houses of worship']

['Outdoor and recreation',
 'Retail',
 'Food and drink',
 'Personal care',
 'Entertainment',
 'Industries',
 'Houses of worship']